## Merge Models

The following code merges the models together creating a new model that you can use directly.

In [ ]:
import os
import sys

# Function to set up the user bin path
def setup_user_bin_path():
    try:
        home_dir = os.environ.get('HOME')
        if home_dir is None:
            raise EnvironmentError("HOME environment variable is not set.")
        
        user_bin_path = os.path.join(home_dir, ".local/bin")
        sys.path.append(user_bin_path)
        print("Updated system path:", sys.path)
    except Exception as e:
        print(f"An error occurred while setting up the user bin path: {e}")

# Call the function
setup_user_bin_path()

In [ ]:
# Function to install required packages--no need to execute this cell twice.
def install_packages():
    packages = [
        "bitsandbytes",
        "transformers",
        "accelerate",
        "peft",
        "trl",
        "tqdm",
        "huggingface_hub"
    ]
    for package in packages:
        try:
            result = os.system(f"{sys.executable} -m pip install -U {package}")
            if result != 0:
                raise Exception(f"Failed to install package: {package}")
        except Exception as e:
            print(f"An error occurred while installing {package}: {e}")

# Call the function
install_packages()

In [ ]:
# Function to handle Hugging Face Hub authentication
def authenticate_huggingface():
    from huggingface_hub import notebook_login, whoami
    try:
        user_info = whoami()
        print('Authorization token already provided')
        print(f"Logged in as: {user_info['name']}")
    except OSError:
        notebook_login()
authenticate_huggingface()

In [ ]:
import os
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import PeftModel

# Function to load and merge models
def load_and_merge_models(base_model_path, merged_model_path, save_directory):
    try:
        # Reload tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(base_model_path)
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_path,
            return_dict=True,
            low_cpu_mem_usage=True,
            torch_dtype=torch.float32,
            offload_buffers=True,
            device_map="auto", 
            trust_remote_code=True,
        )

        # Merge adapter with base model
        model = PeftModel.from_pretrained(base_model, merged_model_path)
        model = model.merge_and_unload()

        # Save the model and tokenizer to the specified directory
        os.makedirs(save_directory, exist_ok=True)
        model.save_pretrained(save_directory)
        tokenizer.save_pretrained(save_directory)

        print("Model and tokenizer have been successfully loaded and merged.")
        return model, tokenizer
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

# Standalone function to execute the model loading and merging
# Note you need to update the merged model path with your username.
def execute_model_loading_and_merging():
    try:
        base_model_path = "meta-llama/Meta-Llama-3-8B"
        merged_model_path = "/home/<your username here>/Gaudi-tutorials/PyTorch/Single_card_tutorials/model_lora_llama3_8B_finetuned"
        save_directory = "models"

        model, tokenizer = load_and_merge_models(base_model_path, merged_model_path, save_directory)

        if model and tokenizer:
            print("Model and tokenizer have been successfully loaded and merged.")
        else:
            print("Failed to load and merge the model and tokenizer.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the standalone function
execute_model_loading_and_merging()